# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import requests

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [59]:
stocks_stats = pd.read_csv("stocks_stats.csv")
stocks_stats_without_NaN = stocks_stats;

In [118]:
stocks_stats_without_NaN = stocks_stats_without_NaN[stocks_stats_without_NaN['P/E'].isna() == False]
stocks_stats_without_NaN = stocks_stats_without_NaN[stocks_stats_without_NaN['P/S'].isna() == False]
stocks_stats_without_NaN = stocks_stats_without_NaN[stocks_stats_without_NaN['P/B'].isna() == False]
stocks_stats_without_NaN = stocks_stats_without_NaN[stocks_stats_without_NaN['EV/EBITDA'].isna() == False]
stocks_stats_without_NaN = stocks_stats_without_NaN[stocks_stats_without_NaN['EV/Revenue'].isna() == False]
stocks_stats_without_NaN

,Symbol,Name,Price,Volume,P/E,PE Percentile,P/B,PB Percentile,P/S,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,RV Score
0,A,Agilent Technologies Inc. Common Stock,140.210,674929,30.97,NaN,7.25,NaN,5.92,NaN,21.36,NaN,6.12,NaN,NaN
5,AAN,Aarons Holdings Company Inc. Common Stock,12.700,349004,26.20,NaN,0.62,NaN,0.21,NaN,1.91,NaN,0.56,NaN,NaN
7,AAON,AAON Inc. Common Stock,90.285,78945,65.19,NaN,8.28,NaN,5.70,NaN,30.20,NaN,5.55,NaN,NaN
8,AAP,Advance Auto Parts Inc.,137.650,480281,16.79,NaN,3.07,NaN,0.76,NaN,11.80,NaN,1.04,NaN,NaN
9,AAPL,Apple Inc. Common Stock,144.485,19134996,25.03,NaN,41.11,NaN,6.16,NaN,18.68,NaN,6.17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715,ZM,Zoom Video Communications Inc. Class A Common ...,69.350,1758193,204.76,NaN,3.28,NaN,4.82,NaN,45.80,NaN,3.42,NaN,NaN
3718,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,24.005,7674522,21.36,NaN,2.63,NaN,3.95,NaN,2.44,NaN,0.56,NaN,NaN
3720,ZTS,Zoetis Inc. Class A Common Stock,168.160,318989,37.32,NaN,17.63,NaN,9.76,NaN,24.58,NaN,10.17,NaN,NaN
3721,ZUMZ,Zumiez Inc. Common Stock,23.230,81291,10.85,NaN,1.19,NaN,0.46,NaN,6.35,NaN,0.55,NaN,NaN


In [131]:
stocks_less_15 = stocks_stats_without_NaN[stocks_stats_without_NaN['Price'] <= 15]
stocks_less_10 = stocks_stats_without_NaN[stocks_stats_without_NaN['Price'] <= 10]

stocks_greater_10 = stocks_stats_without_NaN[(stocks_stats_without_NaN['Price'] >= 10) &
                                            (stocks_stats_without_NaN['Price'] <= 30)]

stocks_less_5 = stocks_stats_without_NaN[stocks_stats_without_NaN['Price'] <= 5]
stocks_less_3 = stocks_stats_without_NaN[stocks_stats_without_NaN['Price'] <= 3]

stocks_30_50 = stocks_stats_without_NaN[(stocks_stats_without_NaN['Price'] >= 30) &
                                            (stocks_stats_without_NaN['Price'] <= 50)]

In [120]:
stocks_greater_10

,Symbol,Name,Price,Volume,P/E,PE Percentile,P/B,PB Percentile,P/S,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,RV Score
5,AAN,Aarons Holdings Company Inc. Common Stock,12.700,349004,26.20,NaN,0.62,NaN,0.21,NaN,1.91,NaN,0.56,NaN,NaN
38,ACI,Albertsons Companies Inc. Class A Common Stock,19.990,2396675,8.04,NaN,13.82,NaN,0.13,NaN,4.40,NaN,0.29,NaN,NaN
39,ACIW,ACI Worldwide Inc. Common Stock,26.015,194138,20.81,NaN,2.43,NaN,2.07,NaN,9.40,NaN,2.72,NaN,NaN
52,ACU,Acme United Corporation. Common Stock,24.430,908,15.94,NaN,1.11,NaN,0.48,NaN,11.51,NaN,0.79,NaN,NaN
81,AEO,American Eagle Outfitters Inc. Common Stock,14.460,2748014,23.16,NaN,1.81,NaN,0.58,NaN,9.50,NaN,0.88,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3710,ZI,ZoomInfo Technologies Inc Common Stock,24.370,936785,151.06,NaN,4.30,NaN,8.88,NaN,31.94,NaN,9.60,NaN,NaN
3713,ZIP,ZipRecruiter Inc. Class A Common Stock,16.940,143523,33.35,NaN,65.49,NaN,2.28,NaN,16.46,NaN,2.06,NaN,NaN
3718,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,24.005,7674522,21.36,NaN,2.63,NaN,3.95,NaN,2.44,NaN,0.56,NaN,NaN
3721,ZUMZ,Zumiez Inc. Common Stock,23.230,81291,10.85,NaN,1.19,NaN,0.46,NaN,6.35,NaN,0.55,NaN,NaN


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [143]:
rv_dataframe = stocks_stats_without_NaN
rv_dataframe = rv_dataframe.reset_index().drop(columns='index')
metrics = {
    'P/E':'PE Percentile',
    'P/B':'PB Percentile',
    'P/S':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Revenue':'EV/Revenue Percentile'
}
for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] \
            = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
        
rv_dataframe

,Symbol,Name,Price,Volume,P/E,PE Percentile,P/B,PB Percentile,P/S,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,RV Score
0,A,Agilent Technologies Inc. Common Stock,140.210,674929,30.97,72.884097,7.25,87.412399,5.92,87.493261,21.36,87.008086,6.12,85.902965,NaN
1,AAN,Aarons Holdings Company Inc. Common Stock,12.700,349004,26.20,65.929919,0.62,5.768194,0.21,4.016173,1.91,10.458221,0.56,15.256065,NaN
2,AAON,AAON Inc. Common Stock,90.285,78945,65.19,90.134771,8.28,89.326146,5.70,86.846361,30.20,93.746631,5.55,83.450135,NaN
3,AAP,Advance Auto Parts Inc.,137.650,480281,16.79,44.150943,3.07,63.584906,0.76,25.849057,11.80,57.681941,1.04,30.970350,NaN
4,AAPL,Apple Inc. Common Stock,144.485,19134996,25.03,63.773585,41.11,98.706199,6.16,88.301887,18.68,81.617251,6.17,86.145553,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,ZM,Zoom Video Communications Inc. Class A Common ...,69.350,1758193,204.76,97.466307,3.28,66.576819,4.82,83.477089,45.80,97.142857,3.42,70.000000,NaN
1851,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,24.005,7674522,21.36,56.442049,2.63,56.495957,3.95,78.921833,2.44,12.048518,0.56,15.256065,NaN
1852,ZTS,Zoetis Inc. Class A Common Stock,168.160,318989,37.32,78.733154,17.63,96.307278,9.76,94.555256,24.58,90.296496,10.17,93.315364,NaN
1853,ZUMZ,Zumiez Inc. Common Stock,23.230,81291,10.85,27.708895,1.19,20.134771,0.46,14.312668,6.35,28.409704,0.55,14.770889,NaN


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [144]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Symbol,Name,Price,Volume,P/E,PE Percentile,P/B,PB Percentile,P/S,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,RV Score
0,A,Agilent Technologies Inc. Common Stock,140.210,674929,30.97,72.884097,7.25,87.412399,5.92,87.493261,21.36,87.008086,6.12,85.902965,84.140162
1,AAN,Aarons Holdings Company Inc. Common Stock,12.700,349004,26.20,65.929919,0.62,5.768194,0.21,4.016173,1.91,10.458221,0.56,15.256065,20.285714
2,AAON,AAON Inc. Common Stock,90.285,78945,65.19,90.134771,8.28,89.326146,5.70,86.846361,30.20,93.746631,5.55,83.450135,88.700809
3,AAP,Advance Auto Parts Inc.,137.650,480281,16.79,44.150943,3.07,63.584906,0.76,25.849057,11.80,57.681941,1.04,30.970350,44.447439
4,AAPL,Apple Inc. Common Stock,144.485,19134996,25.03,63.773585,41.11,98.706199,6.16,88.301887,18.68,81.617251,6.17,86.145553,83.708895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,ZM,Zoom Video Communications Inc. Class A Common ...,69.350,1758193,204.76,97.466307,3.28,66.576819,4.82,83.477089,45.80,97.142857,3.42,70.000000,82.932615
1851,ZTO,ZTO Express (Cayman) Inc. American Depositary ...,24.005,7674522,21.36,56.442049,2.63,56.495957,3.95,78.921833,2.44,12.048518,0.56,15.256065,43.832884
1852,ZTS,Zoetis Inc. Class A Common Stock,168.160,318989,37.32,78.733154,17.63,96.307278,9.76,94.555256,24.58,90.296496,10.17,93.315364,90.641509
1853,ZUMZ,Zumiez Inc. Common Stock,23.230,81291,10.85,27.708895,1.19,20.134771,0.46,14.312668,6.35,28.409704,0.55,14.770889,21.067385


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [145]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(inplace=True, drop=True)
# rv_dataframe = rv_dataframe[:50]
rv_dataframe = rv_dataframe[rv_dataframe['Volume'] > 100000]

In [135]:
price_30_to_50 = rv_dataframe
# price_10_To_30 = rv_dataframe

In [127]:
rv_dataframe.to_csv('Mar_3_pred.csv', index=False)

In [136]:
price_30_to_50

,Symbol,Name,Price,Volume,P/E,PE Percentile,P/B,PB Percentile,P/S,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,RV Score
0,JXN,Jackson Financial Inc. Class A Common Stock,41.5900,632419,0.55,0.346021,0.39,0.346021,0.24,3.114187,-2.17,3.114187,0.16,2.941176,1.972318
1,PBF,PBF Energy Inc. Class A Common Stock,46.7100,670916,2.05,2.422145,1.23,13.840830,0.13,1.038062,1.50,6.574394,0.14,1.903114,5.155709
2,X,United States Steel Corporation Common Stock,30.8800,2312382,3.34,3.460208,0.68,2.422145,0.40,9.342561,1.80,6.920415,0.36,6.055363,5.640138
3,TX,Ternium S.A. Ternium S.A. American Depositary ...,44.4901,104218,4.90,8.650519,0.73,3.114187,0.53,15.570934,1.92,7.266436,0.39,6.574394,8.235294
5,WBA,Walgreens Boots Alliance Inc. Common Stock,34.9925,1409146,5.98,10.380623,1.48,23.529412,0.23,2.768166,-22.91,1.730104,0.47,8.477509,9.377163
6,KBH,KB Home Common Stock,34.6000,358430,3.84,5.536332,0.80,5.536332,0.45,11.245675,4.11,13.494810,0.65,15.570934,10.276817
7,MDC,M.D.C. Holdings Inc. Common Stock,36.1200,143074,4.82,8.131488,0.87,8.131488,0.47,11.937716,3.87,12.456747,0.54,11.591696,10.449827
10,TMHC,Taylor Morrison Home Corporation Common Stock,35.3850,166842,3.97,5.882353,0.84,6.747405,0.51,14.878893,3.88,12.802768,0.70,17.647059,11.591696
14,GM,General Motors Company Common Stock,38.6800,4063152,6.32,11.072664,0.80,5.536332,0.36,7.266436,5.80,20.761246,0.88,25.778547,14.083045
16,WRK,Westrock Company Common Stock,31.9559,379068,10.10,25.259516,0.69,2.768166,0.38,7.958478,5.83,21.107266,0.80,23.010381,16.020761


In [148]:
All = rv_dataframe

In [152]:
All[50:100]

,Symbol,Name,Price,Volume,P/E,PE Percentile,P/B,PB Percentile,P/S,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Revenue,EV/Revenue Percentile,RV Score
111,PBR,Petroleo Brasileiro S.A.- Petrobras Common Stock,10.9800,17821137,2.12,3.153639,1.05,16.495957,0.61,20.592992,1.73,9.757412,0.97,29.164420,15.832884
125,ASX,ASE Technology Holding Co. Ltd. American Depos...,7.2950,1887640,7.88,19.649596,1.56,32.398922,0.71,23.719677,0.14,6.280323,0.03,2.533693,16.916442
127,VLO,Valero Energy Corporation Common Stock,139.1600,1138475,4.54,9.164420,2.08,45.956873,0.30,7.574124,3.09,14.231806,0.32,7.870620,16.959569
129,BTU,Peabody Energy Corporation Common Stock,27.6042,710049,3.36,5.876011,1.24,22.075472,0.88,30.512129,1.77,10.000000,0.62,17.169811,17.126685
131,MPC,Marathon Petroleum Corporation Common Stock,128.9100,937805,4.42,8.760108,1.99,44.070081,0.36,10.107817,2.86,13.369272,0.40,9.811321,17.223720
133,GM,General Motors Company Common Stock,38.6800,4063152,6.32,15.013477,0.80,9.811321,0.36,10.107817,5.80,25.795148,0.88,25.606469,17.266846
135,VET,Vermilion Energy Inc. Common (Canada),13.6700,647392,2.50,3.881402,1.06,16.819407,0.87,30.161725,1.49,9.272237,0.91,26.792453,17.385445
140,AVIR,Atea Pharmaceuticals Inc. Common Stock,3.3950,151033,11.38,28.894879,0.49,3.665768,1.70,52.398922,-11.09,1.994609,-1.77,0.646900,17.520216
143,EQNR,Equinor ASA,31.0900,1099067,3.38,5.983827,1.76,37.789757,0.65,21.805930,0.96,8.544474,0.56,15.256065,17.876011
151,TSN,Tyson Foods Inc. Common Stock,58.5250,1066514,8.80,21.832884,1.07,17.088949,0.40,11.725067,6.08,27.142857,0.54,14.366577,18.431267
